# LangChain Weekend Planner with Azure AI Telemetry

Welcome! This notebook demonstrates how to instrument a LangChain agent to emit GenAI-compliant spans using Azure AI telemetry.

## 🎯 What You'll Build

By the end of this notebook, you'll have:
- ✅ Built a LangChain v1 agent with custom tools
- ✅ Instrumented the agent with Azure AI OpenTelemetry tracer
- ✅ Captured GenAI-compliant spans for LangChain workflows
- ✅ Exported telemetry to Azure Monitor Application Insights

## 💡 What You'll Learn

- How to instrument LangChain agents with OpenTelemetry
- How to use `langchain-azure-ai[opentelemetry]` for automatic telemetry
- How LangChain telemetry differs from native OpenAI Agents
- Best practices for observing LangChain workflows

> **Note**: This uses a weekend planning scenario to demonstrate how OpenTelemetry patterns apply across different agent frameworks (LangChain vs OpenAI Agents).

Ready to instrument LangChain? Let's get started! 🚀

---

## Requirements
Install dependencies before executing the notebook.

```bash
pip install langchain langchain-openai "langchain-azure-ai[opentelemetry]" rich python-dotenv
```

Export the following environment variables (unused entries can remain empty) before running the cells:
- `AZURE_OPENAI_API_KEY`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_VERSION`
- `AZURE_OPENAI_MODEL_NAME`
- `AZURE_AI_FOUNDRY_NAME`
- `AZURE_AOAI_ACCOUNT`
- `AZURE_SUBSCRIPTION_ID`
- `AZURE_RESOURCE_GROUP`
- `AZURE_AISEARCH_ENDPOINT`
- `AZURE_AISEARCH_INDEX`
- `AZURE_AISEARCH_RESOURCE_GROUP`
- `APPLICATION_INSIGHTS_CONNECTION_STRING`

The labs rely solely on this set of variables.


### Step 1: Import dependencies and tracer helpers
Load LangChain, the Azure tracer callback, and supporting utilities while configuring logging and environment handling.


In [ ]:
from __future__ import annotations

import logging
import os
import random
from datetime import datetime

from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain_core.tools import tool
from langchain_openai import AzureChatOpenAI
from rich import print
from rich.logging import RichHandler

from langchain_azure_ai.callbacks.tracers import AzureAIOpenTelemetryTracer

load_dotenv(override=True)

logging.basicConfig(level=logging.WARNING, format="%(message)s", datefmt="[%X]", handlers=[RichHandler()])
LOGGER = logging.getLogger("weekend_planner")
LOGGER.setLevel(logging.INFO)

MODEL_NAME = os.environ.get("AZURE_OPENAI_MODEL_NAME") or "gpt-4o-mini"


### Step 2: Configure the chat model and tracer
This cell selects GitHub Models or Azure OpenAI based on environment variables and prepares the Azure OpenTelemetry tracer.


In [ ]:
TRACER = AzureAIOpenTelemetryTracer(
    connection_string=os.environ.get("APPLICATION_INSIGHTS_CONNECTION_STRING"),
    enable_content_recording=True,
    name="Weekend Planner Agent",
)

MODEL = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2024-05-01-preview"),
    azure_deployment=MODEL_NAME,
)
print("Model configured with model:", MODEL_NAME)


### Step 3: Define tools and initialise the agent
Create the weather, activities, and date tools, then assemble the agent so each invocation can emit GenAI spans.


In [ ]:
@tool
def get_weather(city: str, date: str) -> dict:
    """Returns weather data for a given city and date."""

    LOGGER.info("Getting weather for %s on %s", city, date)
    if random.random() < 0.05:
        return {"temperature": 72, "description": "Sunny"}
    return {"temperature": 60, "description": "Rainy"}


@tool
def get_activities(city: str, date: str) -> list:
    """Returns a list of activities for a given city and date."""

    LOGGER.info("Getting activities for %s on %s", city, date)
    return [
        {"name": "Hiking", "location": city},
        {"name": "Beach", "location": city},
        {"name": "Museum", "location": city},
    ]


@tool
def get_current_date() -> str:
    """Gets the current date from the system and returns as YYYY-MM-DD."""

    LOGGER.info("Getting current date")
    return datetime.now().strftime("%Y-%m-%d")


AGENT = create_agent(
    model=MODEL,
    system_prompt=(
        "You help users plan their weekends and choose the best activities for the given weather. "
        "If an activity would be unpleasant in the weather, avoid suggesting it. "
        "Always include the relevant weekend date in your response."
    ),
    tools=[get_weather, get_activities, get_current_date],
)
print("Agent ready. Tools:", [tool.name for tool in [get_weather, get_activities, get_current_date]])

### Step 4: Invoke the agent
Call the agent with the Azure tracer in the config to emit `invoke_agent` and tool spans. Inspect the notebook output and your telemetry backend before iterating further.


In [ ]:
response = AGENT.invoke(
    {"messages": [{"role": "user", "content": "Hi, what can I do this weekend in San Francisco?"}]},
    config={"callbacks": [TRACER]},
)
latest_message = response["messages"][-1]
print(latest_message.content)